In [8]:
### Importing necessary libraries

import arcpy
import os
import pandas as pd
from arcgis.gis import GIS
from datetime import date
import requests
import importlib
import urllib3
import sys
import config
import openpyxl as pxl
from openpyxl.utils.dataframe import dataframe_to_rows
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print(urllib3.__version__)
import utils

importlib.reload(utils)
importlib.reload(config)

print("Done importing libraries...")

1.26.16
Done importing libraries...


In [79]:
### Setting dates for name

today = date.today()
date_ = today.strftime("%Y%m%d")

In [80]:
### Log in to org

gis_source = utils.loggin_agol("config.py")

C:\Users\omar9125\OneDrive - Esri\Working_Files_7292022\DoE_Scriptign_5172022\config.py
Config File found and will continue!
Loging in as odelariva into https://red-ps-fwp-prtl.esri.com/portal/home/! Please wait...
Success! Logged into GIS @ https://red-ps-fwp-prtl.esri.com/portal/home/ version:10.3 as odelariva!


In [81]:
### Get Token from GIS Source
### If issues getting a token, please manually input your token here. 

token_ = utils.request_token(gis_source)
#token_ = 'YOUR_TOKEN_HERE'

Will get tokenf from the source GIS here: GIS @ https://red-ps-fwp-prtl.esri.com/portal/home/ version:10.3
Here is your current token: 
t6FATEG0qS1OuQEO-Gv7DZ4pgJVrYG5PWggxYJmGMyUtPNB7AhHhWu1QuNDp25uQiJ2ttu3Ti1GVCEsAQIvO44LI6XJqO6DnQKLFY7YTZvc-7uJzrZhmHshop0UHnUAi4qLFanQcVmIfRiySS3DYBkxHgvPq9VSxxQCZxUVckFj4hzGNKqsNenDNjzkWs4dkLSBVH2CX1u4ZAPW45zZg9xCA-DKi4kXq2iq7SZOPfPI.


In [82]:
### Log in to portal and start the query

params = {'f': 'json', 'token': token_}
source_users = gis_source.users.search(query='', sort_field='username', sort_order='asc', max_users=10000, outside_org=False, exclude_system=True)
print('Logged in to {}'.format(gis_source))

Logged in to GIS @ https://red-ps-fwp-prtl.esri.com/portal/home/ version:10.3


In [83]:
### Creating alist of all feature services, max_items_returned to 1000

MAX_ITEMS_RETURNED = 1000
item_list = gis_source.content.search(query='owner:*', item_type='Feature Service', max_items=MAX_ITEMS_RETURNED)

print("We have {} feature services to process.".format(len(item_list)))

We have 285 feature services to process.


In [84]:
item_list = utils.pop_empty_urls(item_list)
item_list = utils.pop_gdb_urls(item_list)
item_list = utils.pop_repeated_urls(item_list)
item_list = utils.clean_urls(item_list)

Popping out empty urls...
Finished running the script!
Finished cleaning URL's ending with GDB.
We found 0 repeated URLs and they have been removed.
Finished cleaning repeated URL's...
Clearing numeric URL's
Finished clearning numeric URL's


In [85]:
### Starting the iterations here...

print("Starting iterations now...")
utils.lil_spacer()

dict_ = {}
url_dict_ = {}

for item_ in item_list:
    
    print(f"Iterating through: {item_.title}")
    print(f"Inital URL: {item_.url}")
    try:
        error_      = False
        status_     = False
        all_errors_ = False
        
        current_url = item_.url.replace('rest','admin').replace('/MapServer','.MapServer') + '/iteminfo/manifest/manifest.json'
        print(f"Updated URL: {current_url}")
        
        response = requests.post(current_url, params = params, verify = False)
        response_json = response.json()
        status_, error_ = utils.check_status_error(response_json, error_, status_)
        
        if status_ or error_:
            print("Found an error when pulling the JSON String MapServer to MapServer")
            all_errors_ = True
    
        if all_errors_:
            print("Did not append to dictionary!")
            utils.lil_spacer()
            raise Error
        else:
            url_dict_[item_.title] = current_url
            dict_[item_] = response_json
            print(f"Appended the following: {response_json}")  
            utils.lil_spacer()
        
    except:
        print("JSON Response Failed when using /MapServer to .MapServer, will try another path... " )
        
        error_      = False
        status_     = False
        all_errors_ = False
        
        try:
            current_url = item_.url.replace('rest','admin').replace('/FeatureServer','.MapServer') + '/iteminfo/manifest/manifest.json'
            print(f"Updated URL: {current_url}")

            response = requests.post(current_url, params = params, verify = False)
            response_json = response.json()
            status_, error_ = utils.check_status_error(response_json, error_, status_)

            if status_ or error_:
                print("Found error when pulling the JSON string FeatureServer to MapServer")
                all_errors_ = True

            if all_errors_:
                print("Did not append to dictionary!")
                utils.lil_spacer()
                raise Error
            else:
                url_dict_[item_.title] = current_url
                dict_[item_] = response_json
                print(f"Appended the following to dictionary: ")
                print(response_json)
                utils.lil_spacer()
        
        except:
            print("JSON Response Failed when using /FeatureServer to .MapServer. Both attemps failed will not append. ")
            pass 
    
    utils.lil_dashy()
    utils.lil_spacer()
        
print("Finished iterating through all items!")          

Starting iterations now...


Iterating through: FB Scene Test 5_WSL3
Inital URL: https://red-ps-fwp-prtl.esri.com/hosted/rest/services/Hosted/FB_Scene_Test_5_WSL3/FeatureServer
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/FB_Scene_Test_5_WSL3/FeatureServer/iteminfo/manifest/manifest.json
Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/FB_Scene_Test_5_WSL3.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SER

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/EsriCampus_BuildingE_3122021_WFL1.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\omar9

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/FB_WebScene_8252021_WSL1.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\omar9125\\Desk

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/LAWA_3D_Scene_V3_WSL2.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD_UTF8="00022e6835456d545052626e6d7062422f587074757373356f564d626d307772303777466d70384c796371687742303d2a00";ENCRYPTED_PASSWORD="00022e686b377875764b344b4e7247487038557134525344424436412f7264385478612b573830364d45574f68656b3d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hs

Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD_UTF8="00022e6835456d545052626e6d7062422f587074757373356f564d626d307772303777466d70384c796371687742303d2a00";ENCRYPTED_PASSWORD="00022e686b377875764b344b4e7247487038557134525344424436412f7264385478612b573830364d45574f68656b3d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\omar9125\\OneDrive - Esri\\Working_Files_7292022\\Denton_02062023\\Denton_262023\\Denton_Workshop_01.gdb', 'onServerName': 'sde.DEFAULT (RED-PS-FWP-RDS.ESRI.COM)', 'onPremisePath': '', 'datasets': [{'onServerName': 'Stairs_POI'}]}], 'reso

...

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/0813_Indoors_3D_WSL5.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\tif11668\\Desktop\

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/0813_Indoors_3D_WSL2.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\tif11668\\Desktop\

Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Projects\\Procon\\Procon.gdb', 'onServerName': 'sde.DEFAULT (RED-PS-FWP-RDS.ESRI.COM)', 'onPremisePath': '', 'datasets': [{'onServerName': 'ExteriorShell_SHCA0343ACentralR19_BIMFileT'}]}], 'resources': [{'onPremisePath': 'C:\\Projects\\Procon\\Procon.aprx', 'clientName': 'Gsnipes2', 'serverPath': 'D:\\arcgisserver\\directories\\arcgissystem\\arcgisinput\\Hosted\\TestSce

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/0813_Indoors_3D_WSL3.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\tif11668\\Desktop\

Status: True
Error: False
Found an error when pulling the JSON String MapServer to MapServer
Did not append to dictionary!


JSON Response Failed when using /MapServer to .MapServer, will try another path... 
Updated URL: https://red-ps-fwp-prtl.esri.com/hosted/admin/services/Hosted/hq_space.MapServer/iteminfo/manifest/manifest.json
Status: False
Error: False
Appended the following to dictionary: 
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\maya9592\\OneDrive - Esri\\Doc

In [86]:
for k, v in dict_.items():
    print(k.title)
    print(v)
    utils.lil_spacer()

FB Scene Test 5_WSL3
{'databases': [{'byReference': False, 'onServerWorkspaceFactoryProgID': 'esriDataSourcesGDB.SdeWorkspaceFactory', 'onServerConnectionString': 'ENCRYPTED_PASSWORD="00022e684e426a3853436e396664336377326c6a374e584e706a49676d70666655705241613750733173304d4e6b303d2a00";SERVER="RED-PS-FWP-RDS.ESRI.COM";INSTANCE="sde:postgresql:RED-PS-FWP-RDS.ESRI.COM,9876";DBCLIENT="postgresql";DB_CONNECTION_PROPERTIES="RED-PS-FWP-RDS.ESRI.COM,9876";DATABASE="db_hw5r1";USER="hsu_jccuc";VERSION="sde.DEFAULT";AUTHENTICATION_MODE="DBMS"', 'onPremiseConnectionString': 'DATABASE=C:\\Users\\maya9592\\OneDrive - Esri\\Documents\\Projects\\Facebook\\Project Data\\3D\\FacebookBIMtoGIS.gdb', 'onServerName': 'sde.DEFAULT (RED-PS-FWP-RDS.ESRI.COM)', 'onPremisePath': '', 'datasets': [{'onServerName': 'Units3D_9'}]}], 'resources': [{'onPremisePath': 'C:\\Users\\maya9592\\OneDrive - Esri\\Documents\\Projects\\Facebook\\Pro Projects\\3D_Publishing\\3D_Publishing.aprx', 'clientName': 'MOBrien', 'serverPa

In [87]:
for k, v in dict_.items():
    print(f"Processing {k.title}")
    
    on_ser_con = v['databases'][0]['onPremiseConnectionString'].split(';')
    on_ser_con_len = len(on_ser_con)
    
    on_prem_con = v['databases'][0]['onPremiseConnectionString'].split(';')
    on_prem_con_len = len(on_prem_con)

    
    if on_ser_con_len == 9 or on_ser_con_len == 10:
        print(f"On Server Connection: {on_ser_con_len}")
        print(on_ser_con)
    utils.lil_spacer()
    
    if on_prem_con_len == 9 or on_prem_con_len == 10:
        print(f"On Premise Connection: {on_prem_con_len}")
        print(on_prem_con)
    
    utils.lil_bugger()

Processing FB Scene Test 5_WSL3


**************************************************
Processing 0817_Indoors_3D_WSL5


**************************************************
Processing LAWA 3D Scene V1_WFL1


**************************************************
Processing EsriForDOJCRT
On Server Connection: 9
['ENCRYPTED_PASSWORD=00022e6856654a565a706f68386d462b7268442f6e636a414875376e46545144654c7557764155384e6c6157344b493d2a00', 'SERVER=red-ps-fwp-egdb', 'INSTANCE=sde:sqlserver:red-ps-fwp-egdb', 'DBCLIENT=sqlserver', 'DB_CONNECTION_PROPERTIES=red-ps-fwp-egdb', 'DATABASE=dojtest', 'USER=dojtestuser', 'AUTHENTICATION_MODE=DBMS', 'BRANCH=sde.DEFAULT']


On Premise Connection: 9
['ENCRYPTED_PASSWORD=00022e6856654a565a706f68386d462b7268442f6e636a414875376e46545144654c7557764155384e6c6157344b493d2a00', 'SERVER=red-ps-fwp-egdb', 'INSTANCE=sde:sqlserver:red-ps-fwp-egdb', 'DBCLIENT=sqlserver', 'DB_CONNECTION_PROPERTIES=red-ps-fwp-egdb', 'DATABASE=dojtest', 'USER=dojtestuser', 'AUTHENTICATION_MODE=D

In [88]:
main_list = []
service_counter = 0

hosted_list = []

print('Iterating through services...')
for k, v in dict_.items():
    service_counter_ = 0
    list_            = []
    hosted_          = []
    len_url          = len(item_.url)
    
    print(f"Processing {k.title} feature service...")
    
    print(k.title)
    print(k.owner)
    print(k.url)
    
    list_.append(k.title)
    list_.append(k.owner)
    list_.append(k.url)
    
    hosted_.append(k.title)
    hosted_.append(k.owner)
    hosted_.append(k.url)

    ### Hosted difference can go here TODO
    
    if 'rest/services/Hosted' in k.url:
        hosted_ = []
        hosted_.append(k.title)
        hosted_.append(k.owner)
        hosted_.append(k.url)
        hosted_db = v['databases'][0]['onPremiseConnectionString']
        hosted_.append(hosted_db)
    
        hosted_list.append(hosted_)
        
        del hosted_
        
    else:
        try:
            list_.append(url_dict_[k.title])
        except:
            list_.append('Missing URL')

        utils.lil_spacer()

        ### deal with idtem with no databases here TODO 

        on_ser_con = v['databases'][0]['onServerConnectionString'].split(';')
        on_ser_con_len = len(on_ser_con)

        print(f"Your onServerConnection String is length of: {on_ser_con_len}")

        if v['databases'][0]['onServerConnectionString'][-4:] == '.gdb':
            print("Response ends with a GDB, will not append onServer data...")

            list_.append('Service ends in GDB')
            list_.append('Service ends in GDB')
            list_.append('Service ends in GDB')

        elif on_ser_con_len == 1:
            list_.append(on_ser_con)
            list_.append(" ")
            list_.append(" ")

        else:
            e_ = 0
            
            if on_ser_con_len == 10:
                e_ = 1
            
            print("Reponse does not end with GDB, checking onServerConnection string...")
            split_ = v['databases'][0]['onServerConnectionString'].split(';')
            print(split_)

            if split_[2 + e_] is None or split_[2 + e_] == '':
                list_.append("No onServerConnection Instance")
                print('No onServerConnectionn Instance')
            else:
                list_.append(split_[2 + e_])
                print(f"Appended ON_SER_INSTANCE: {split_[2 + e_]}")

            if split_[3 + e_] is None or split_[3 + e_] == '':
                list_.append("No onServerConnection DB Client")
                print('No onServerConnection DB Client')
            else:
                list_.append(split_[3 + e_])
                print(f"Appended ON_SER_DB_CLIENT: {split_[3 + e_]}")

            
            if split_[4 + e_] is None or split_[4 + e_] == '':
                list_.append("No onServerConnection DB Connection")
                print('No onServerConnection DB Connection')
            else:
                list_.append(split_[4 + e_])
                print(f"Appended ON_SER_DB_CONNECT: {split_[4 + e_]}")

            if split_[5 + e_] is None or split_[5 + e_] == '':
                list_.append("No onServerConnection Instance")
                print('No onServerConnection Instance')
            else:
                list_.append(split_[5 + e_])
                print(f"Appended ON_SER_PROJ: {split_[5 + e_]}")
                
            if split_[6 + e_] is None or split_[6 + e_] == '':
                list_.append("No onServerConnection User")
                print('No onServerConnection User')
            else:
                list_.append(split_[6 + e_])
                print(f"Appended ON_SER_USER: {split_[6 + e_]}")
                
            if split_[7 + e_][:4] == 'AUTH':
                list_.append(split_[7 + e_])
                print(f"Appended ON_SER_AUTH: {split_[7 + e_]}")
                
                list_.append(split_[8 + e_])
                print(f"Appended ON_SER_VERSION: {split_[8 + e_]}")
            elif split_[7 + e_][:4] == 'BRAN' or split_[7 + e_][:4] == 'VERS':
                list_.append(split_[8 + e_])
                print(f"Appended ON_SER_AUTH: {split_[8 + e_]}")

                list_.append(split_[7 + e_])
                print(f"Appended ON_SER_VERSION: {split_[7 + e_]}")
            else:
                list_.append(' ')
                list_.append(' ')
                
            del split_

        utils.lil_spacer()

        on_prem_con = v['databases'][0]['onPremiseConnectionString'].split(';')
        on_prem_con_len = len(on_prem_con)

        print(f"Your onPremiseConnection String is length of: {on_prem_con_len}")
        print(on_prem_con)

        if v['databases'][0]['onPremiseConnectionString'][-4:] == '.gdb':
            print("Response ends with a GDB, will not append onPremise data...")

            list_.append('Service ends in GDB')
            list_.append('Service ends in GDB')
            list_.append('Service ends in GDB')
            utils.lil_spacer()

        elif on_prem_con_len == 1:
            list_.append(on_prem_con)
            list_.append(" ")
            list_.append(" ")

        else:
            e_ = 0
            if on_prem_con_len == 10:
                e_ = 1
            
            print("Reponse does not end with GDB, checking onPremiseConnection string...")
            split_ = v['databases'][0]['onPremiseConnectionString'].split(';')

            if split_[2 + e_] is None or split_[2 + e_] == '':
                list_.append("No onPremiseConnection Instance")
                print('No onPremiseConnection Instance')
            else:
                list_.append(split_[2 + e_])
                print(f"Appended ON_PREM_INSTANCE: {split_[2 + e_]}")

            if split_[3 + e_] is None or split_[3 + e_] == '':
                list_.append("No onPremiseConnection DB Client")
                print('No onPremiseConnection DB Client')
            else:
                list_.append(split_[3 + e_])
                print(f"Appended ON_PREM_DB_CLIENT: {split_[3 + e_]}")

            
            if split_[4 + e_] is None or split_[4 + e_] == '':
                list_.append("No onPremiseConnection DB Connection")
                print('No onPremiseConnection DB Connection')
            else:
                list_.append(split_[4 + e_])
                print(f"Appended ON_PREM_DB_CONNECT: {split_[4 + e_]}")

            if split_[5 + e_] is None or split_[5 + e_] == '':
                list_.append("No onPremiseConnection Instance")
                print('No onPremiseConnection Instance')
            else:
                list_.append(split_[5 + e_])
                print(f"Appended ON_PREM_PROJ: {split_[5 + e_]}")
                
            if split_[6 + e_] is None or split_[6 + e_] == '':
                list_.append("No onPremiseConnection User")
                print('No onPremiseConnection User')
            else:
                list_.append(split_[6 + e_])
                print(f"Appended ON_PREM_USER: {split_[6 + e_]}")
                
            if split_[7 + e_][:4] == 'AUTH':
                list_.append(split_[7 + e_])
                print(f"Appended ON_PREM_AUTH: {split_[7 + e_]}")
                
                list_.append(split_[8 + e_])
                print(f"Appended ON_PREM_VERSION: {split_[8 + e_]}")
                
            elif split_[7 + e_][:4] == 'BRAN' or split_[7 + e_][:4] == 'VERS':
                list_.append(split_[8 + e_])
                print(f"Appended ON_PREM_AUTH: {split_[8 + e_]}")

                list_.append(split_[7 + e_])
                print(f"Appended ON_PREM_VERSION: {split_[7 + e_]}")
            else:
                list_.append(' ')
                list_.append(' ')
             
            del split_
            utils.lil_spacer()      

        print(f"Getting the number of the {k.title} databases...")

        db_len = len(v['databases'])
        print(f"{k.title} has {db_len} of databases...")
        for i in range(db_len):
            #print(v['databases'][i]['datasets'])
            ds_len = len(v['databases'][i]['datasets'])
            for j in range(ds_len):
                service_ = v['databases'][i]['datasets'][j]['onServerName']
                print(f"On ServerName/Services: {service_}")
                list_.append(service_)
                service_counter_ = service_counter_ + 1

        main_list.append(list_)
        hosted_list.append(hosted_)

        if service_counter_ > service_counter:
            service_counter = service_counter_

        del list_, service_counter_

    utils.lil_spacer()
    utils.lil_dashy()
    
print("Tool has finished running!")

Iterating through services...
Processing FB Scene Test 5_WSL3 feature service...
FB Scene Test 5_WSL3
mobrien
https://red-ps-fwp-prtl.esri.com/hosted/rest/services/Hosted/FB_Scene_Test_5_WSL3/FeatureServer


--------------------------------------------------
Processing 0817_Indoors_3D_WSL5 feature service...
0817_Indoors_3D_WSL5
tlim_indoors
https://red-ps-fwp-prtl.esri.com/hosted/rest/services/Hosted/0817_Indoors_3D_WSL5/FeatureServer


--------------------------------------------------
Processing LAWA 3D Scene V1_WFL1 feature service...
LAWA 3D Scene V1_WFL1
odelariva
https://red-ps-fwp-prtl.esri.com/hosted/rest/services/Hosted/LAWA_3D_Scene_V1_WFL1/FeatureServer


--------------------------------------------------
Processing EsriForDOJCRT feature service...
EsriForDOJCRT
thahka
https://red-ps-fwp-prtl.esri.com/server/rest/services/EsriForDOJCRT/FeatureServer


Your onServerConnection String is length of: 9
Reponse does not end with GDB, checking onServerConnection string...
['ENCRYP

In [96]:
### Creating pandas dataframes and corresponding columns

#out_path   = fr'C:\Users\omar9125\OneDrive - Esri\Working_Files_7292022\DoE_Scriptign_5172022\Output_CSV\GIS WebServices Connection {date_}.xlsx'
out_path   = r'YOUR_OUT_PATH'

columns_to_pd = ['TITLE', 'OWNER', 'URL','UPDATED_URL','ON_SER_INSTANCE', 'ON_SER_DB_CLIENT', 'ON_SER_CONNPROP', 'ON_SER_DATABASE', 'ON_SER_USER', 'ON_SER_AUTH', 'ON_SER_BRANCH', 'ON_PREM_INSTANCE','ON_PREM_DB_CLIENT','ON_PREM_DB_CONN', 'ON_PREM_DATABASE', 'ON_PREM_USER', 'ON_PREM_AUTH', 'ON_PREM_VERSION']
hosted_columns = ['TITLE','OWNER','URL','HOSTED DATABASE']

### Setting up extended field names
for ser_ in range(service_counter):
    columns_to_pd.append("Services_" + str(ser_))

### Setting up field names for extended columns
output_df = pd.DataFrame(main_list, columns = columns_to_pd)
hosted_df = pd.DataFrame(hosted_list, columns = hosted_columns)
#output_df = pd.DataFrame(main_list)
#hosted_df = pd.DataFrame(hosted_list)
    

In [97]:
### Creating the output Excel sheet. 

if os.path.exists(out_path):
    os.remove(out_path)
    print("{0} has been deleted.".format(out_path))

output_df.to_excel(out_path, index=False, sheet_name = "Services")
excel_book = pxl.load_workbook(out_path)
excel_book.create_sheet('Hosted')
rows = dataframe_to_rows(hosted_df, index = False)

ws = excel_book["Hosted"]

for r_idx, row in enumerate(rows, 1):
   #print(r_idx)
    for c_idx, value in enumerate(row, 1):
      #  print(c_idx)
        ws.cell(row = r_idx, column = c_idx, value = value)

excel_book.save(out_path)

#output_df.to_csv(out_path)   
print('Excel file created!')

Excel file created!


In [7]:
help(pd)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

